In [14]:
#import libraries
import os
import pandas as pd 
import numpy as np 
import matplotlib as plt
import seaborn as sns 

Matplotlib is building the font cache; this may take a moment.


In [5]:
#import data as csv
df_bp= pd.read_csv('/Users/jonesyy/Downloads/SROP Work/data/06_06_25/all_bps_with_all_regs_apr_2025.csv')

/var/folders/t1/d9jqqs655cs0r3yvpzy6s6280000gn/T/ipykernel_27387/11551485.py:2: DtypeWarning: Columns (4,9,33,34,35,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bp= pd.read_csv('/Users/jonesyy/Downloads/SROP Work/data/06_06_25/all_bps_with_all_regs_apr_2025.csv')


In [8]:
df_bp.columns

Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)', 'nt1',
       'chain_ID_res1', 'res_ID_res1', 'res_index_res1', 'icode_res1', 'nt2',
       'chain_ID_res2', 'res_ID_res2', 'res_index_res2', 'icode_res2',
       'base_pair', 'name', 'Saenger', 'LW', 'DSSR', 'bp_res',
       'distance_hbond_1', 'distance_hbond_2', 'distance_hbond_3',
       'atoms_serNum_hbond_1', 'atoms_serNum_hbond_2', 'atoms_serNum_hbond_3',
       'atoms_ID_hbond_1', 'atoms_ID_hbond_2', 'atoms_ID_hbond_3',
       'distance_hbond_4', 'atoms_serNum_hbond_4', 'atoms_ID_hbond_4',
       'distance_hbond_5', 'distance_hbond_6', 'atoms_serNum_hbond_5',
       'atoms_serNum_hbond_6', 'atoms_ID_hbond_5', 'atoms_ID_hbond_6',
       'distance_hbond_7', 'atoms_serNum_hbond_7', 'atoms_ID_hbond_7',
       'distance_hbond_8', 'distance_hbond_9', 'atoms_serNum_hbond_8',
       'atoms_serNum_hbond_9', 'atoms_ID_hbond_8', 'atoms_ID_hbond_9',
       'distance_hbond_10', 'atoms_serNum_hbond_10', 'atoms_ID_hbond_10'],
      dty

In [9]:
df_bp.shape

(3728509, 49)

In [10]:
df_bp

,PDB_ID,Experimental_Method,Resolution_(Å),nt1,chain_ID_res1,res_ID_res1,res_index_res1,icode_res1,nt2,chain_ID_res2,...,atoms_ID_hbond_7,distance_hbond_8,distance_hbond_9,atoms_serNum_hbond_8,atoms_serNum_hbond_9,atoms_ID_hbond_8,atoms_ID_hbond_9,distance_hbond_10,atoms_serNum_hbond_10,atoms_ID_hbond_10
0,157D,X-RAY DIFFRACTION,1.8,A.A5,A,A,5.0,NaN,B.U20,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,157D,X-RAY DIFFRACTION,1.8,A.A6,A,A,6.0,NaN,B.U19,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,157D,X-RAY DIFFRACTION,1.8,A.A9,A,A,9.0,NaN,B.G16,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,157D,X-RAY DIFFRACTION,1.8,A.C1,A,C,1.0,NaN,B.G24,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,157D,X-RAY DIFFRACTION,1.8,A.C11,A,C,11.0,NaN,B.G14,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3728504,9NLS,ELECTRON MICROSCOPY,3.3,T.G9,T,G,9.0,NaN,T.G45,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3728505,9NLS,ELECTRON MICROSCOPY,3.3,T.U27,T,U,27.0,NaN,T.A43,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3728506,9NLS,ELECTRON MICROSCOPY,3.3,T.U50,T,U,50.0,NaN,T.G64,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3728507,9NLS,ELECTRON MICROSCOPY,3.3,T.U7,T,U,7.0,NaN,T.A13,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_bp['name'].value_counts()

name
WC             1928534
--              951512
Wobble          233908
Sheared         185192
Linker          100123
rHoogsteen       94486
Platform         59282
Imino            34710
~Sheared         34024
~rHoogsteen      28342
rWC              25609
Hoogsteen        22712
~Wobble          17367
Calcutta          3958
~rWobble          3479
~Hoogsteen        3195
rWobble           1488
Metal              588
Name: count, dtype: int64

In [15]:
df_bp['name'].plot(kind='bar')
 

TypeError: no numeric data to plot